In [1]:
# -*- coding: utf-8 -*- #

# Write libraries here #########################################
import matplotlib.patches as pat
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from astropy import units as u
from astropy.cosmology import WMAP9 as cosmo
from astroquery.gama import GAMA
from jupyterthemes import jtplot
from mpl_toolkits.mplot3d import Axes3D
from sklearn import manifold, cluster, preprocessing, mixture
from time import time
from tqdm import tnrange, tqdm_notebook
#%matplotlib inline
%matplotlib notebook
################################################################


# Showing 500 columns and rows #################################
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
################################################################


# NOT CHANGE (The PATH for directory) ##########################
CURRENT_DIR_NAME = os.path.dirname(os.path.abspath(__name__))
PATH_DATA = os.path.join(CURRENT_DIR_NAME, '../data/')
PATH_HIST = os.path.join(CURRENT_DIR_NAME, '../histogram/')
PATH_PLOT = os.path.join(CURRENT_DIR_NAME, '../plots/')
PATH_IMG = os.path.join(CURRENT_DIR_NAME, '../images/')
PATH_HTML = os.path.join(CURRENT_DIR_NAME, '../html/')
PATH_TEX = os.path.join(CURRENT_DIR_NAME, '../for_tex/')
################################################################

#When you make plots, jtplot is better.
jtplot.style(theme='grade3', context='notebook', fscale=3, spines=True, gridlines='--', ticks=False, grid=False)

In [4]:
completed_data = pd.read_pickle(PATH_DATA + 'GAMADataForManifold.pickle')

In [7]:
display(completed_data.describe())

,RA,DEC,FUV_flux,FUV_fluxerr,NUV_flux,NUV_fluxerr,u_flux,u_fluxerr,g_flux,g_fluxerr,r_flux,r_fluxerr,i_flux,i_fluxerr,z_flux,z_fluxerr,X_flux,X_fluxerr,Y_flux,Y_fluxerr,J_flux,J_fluxerr,H_flux,H_fluxerr,K_flux,K_fluxerr,W1_flux,W1_fluxerr,W2_flux,W2_fluxerr,W3_flux,W3_fluxerr,W4_flux,W4_fluxerr,P100_flux,P100_fluxerr,P160_flux,P160_fluxerr,S250_flux,S250_fluxerr,S350_flux,S350_fluxerr,S500_flux,S500_fluxerr,Z_TONRY,NQ,KCORR_FUV,KCORR_NUV,KCORR_U,KCORR_G,KCORR_R,KCORR_I,KCORR_Z,KCORR_Y,KCORR_J,KCORR_H,KCORR_K,PCOEFF_R_1,PCOEFF_R_2,PCOEFF_R_3,PCOEFF_R_4,PCOEFF_R_5,COEFFS_1,COEFFS_2,COEFFS_3,COEFFS_4,COEFFS_5,CHI2,MASS,INTSFH,METS,B300,B1000,FUV_flux_kcorrected,NUV_flux_kcorrected,H_flux_kcorrected,I_flux_kcorrected,K_flux_kcorrected,G_flux_kcorrected,Z_flux_kcorrected,Y_flux_kcorrected,J_flux_kcorrected,R_flux_kcorrected,U_flux_kcorrected,R-K,NUV-R
count,73768.000000,73768.000000,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,7.376800e+04,73768.000000,73768.000000,7.376800e+04,73768.000000,7.376800e+04,73768.000000,7.376800e+04,73768.000000,7.376800e+04,73768.000000,73768.000000,73768.000000,7.376800e+04,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,7.376800e+04,7.376800e+04,7.376800e+04,73768.000000,7.376800e+04,73768.000000,7.376800e+04,7.376800e+04,73768.000000,7.376800e+04,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000,73768.000000
mean,189.651980,0.375954,9.691125e-06,1.433836e-06,1.771798e-05,6.959949e-06,3.941530e-05,5.811074e-06,1.233047e-04,6.644793e-06,2.410331e-04,9.234676e-06,3.432591e-04,1.262331e-05,4.200821e-04,2.667000e-05,-4.725897,-4.726298,5.097796e-04,0.000035,6.000265e-04,0.000032,7.329608e-04,0.000037,7.477631e-04,0.000038,0.000396,0.000068,2.707400e-04,0.000066,0.000687,0.000141,0.001333,0.000921,-134.476986,-134.485920,-134.835816,-134.849297,-145.425209,-145.439338,-145.393815,-145.398589,-145.304460,-145.302563,0.200553,4.123956,0.253080,0.121602,0.428708,0.458146,0.203161,0.062516,0.070858,0.099476,-0.018801,0.004191,-0.330487,-0.001001,1.728377,-6.328902,12.690175,-4.808171,3.833503e-06,1.058281e-09,7.237552e-07,0.000009,1.680758e-07,2.194935,2.750862e+10,4.958471e+10,0.029632,1.950617e-02,0.265940,-17.261736,-18.144607,-22.214019,-21.465056,-21.983333,-20.628491,-21.685446,-21.919099,-21.973418,-21.216586,-19.327046,0.766747,3.071979
std,33.758608,1.356445,7.012375e-05,1.041517e-05,1.183519e-04,7.378952e-05,3.199671e-04,2.849965e-05,1.023210e-03,2.587866e-05,1.833359e-03,3.051119e-05,2.540001e-03,3.919699e-05,3.083389e-03,9.312436e-05,68.551593,68.551566,3.493307e-03,0.000169,4.066153e-03,0.000131,4.841449e-03,0.000115,4.133784e-03,0.000131,0.002416,0.000197,1.440871e-03,0.000228,0.006122,0.000378,0.011975,0.000886,341.008456,341.004835,341.402135,341.396708,352.354585,352.348732,352.309926,352.307952,352.212370,352.213152,0.102923,0.373316,0.526796,0.247942,0.360486,0.337047,0.145953,0.143677,0.107124,0.122149,0.091785,0.067802,0.125817,0.003224,0.682940,6.629421,18.311508,17.458761,2.259973e-05,5.150192e-09,2.927421e-06,0.000057,7.691974e-07,22.868611,4.735939e+10,8.489733e+10,0.007677,4.240422e-02,0.187483,1.608674,1.438503,1.445310,1.316428,1.461947,1.188454,1.354942,1.389135,1.407693,1.252013,1.195957,0.359255,1.280014
min,129.000109,-2.993768,3.332060e-10,5.765690e-09,9.628900e-10,3.340650e-08,1.335430e-08,2.499510e-07,4.235320e-08,1.292490e-07,1.538570e-08,1.932670e-07,3.210630e